In [1]:
import numpy as np
from PIL import Image

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Torchvision
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard.summary import image

In [2]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_data = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms)

#test data

test_data = torchvision.datasets.CIFAR10(
    './data',
    train=False,
    download=True,
    transform=transforms
)

#train loader
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

# test loader
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

In [4]:
image, label = train_data[0]

In [5]:
image.size()

torch.Size([3, 32, 32])

In [6]:
class_name = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # (24, 10, 10) -> (24, 5, 5) -> Flattened to (24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120) # (120)
        self.fc2 = nn.Linear(120, 84) # (84)
        self.fc3 = nn.Linear(84, 10) # (10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Apply conv1 and relu, then max pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply conv2 and relu, then max pooling
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))  # Apply fc1 and relu
        x = F.relu(self.fc2(x))  # Apply fc2 and relu
        x = self.fc3(x)  # Apply fc3 (output layer)
        return x




In [8]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(30):
    print(f'Training epoch {epoch }')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0
Loss: 2.2108
Training epoch 1
Loss: 1.7839
Training epoch 2
Loss: 1.5085
Training epoch 3
Loss: 1.3966
Training epoch 4
Loss: 1.3143
Training epoch 5
Loss: 1.2390
Training epoch 6
Loss: 1.1666
Training epoch 7
Loss: 1.0994
Training epoch 8
Loss: 1.0435
Training epoch 9
Loss: 0.9941
Training epoch 10
Loss: 0.9524
Training epoch 11
Loss: 0.9082
Training epoch 12
Loss: 0.8740
Training epoch 13
Loss: 0.8457
Training epoch 14
Loss: 0.8130
Training epoch 15
Loss: 0.7795
Training epoch 16
Loss: 0.7517
Training epoch 17
Loss: 0.7243
Training epoch 18
Loss: 0.6968
Training epoch 19
Loss: 0.6712
Training epoch 20
Loss: 0.6448
Training epoch 21
Loss: 0.6229
Training epoch 22
Loss: 0.5982
Training epoch 23
Loss: 0.5752
Training epoch 24
Loss: 0.5530
Training epoch 25
Loss: 0.5315
Training epoch 26
Loss: 0.5104
Training epoch 27
Loss: 0.4960
Training epoch 28
Loss: 0.4706
Training epoch 29
Loss: 0.4550


In [10]:
torch.save(net.state_dict(), 'trained_net.pth')

In [11]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [12]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')

Accuracy of the network on the 10000 test images: 68.48%


In [13]:
import torch
import torchvision.transforms as transforms
from PIL import Image

"""Test Images"""

# Definição das transformações
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Função para carregar e pré-processar a imagem
def load_image(image_path):
    image = Image.open(image_path).convert("RGB") # adicionei .convert("RGB") para garantir 3 canais
    image = new_transform(image)
    image = image.unsqueeze(0)  #  dimensão do batch
    return image

# Lista de caminhos das imagens
image_paths = ['test_images/img.png', 'test_images/img2.png']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad(): # --> Desabilita o cálculo de gradientes para inferência
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Predicted class: {class_name[predicted.item()]}')

Predicted class: dog
Predicted class: plane
